<html>
<body>
    <center> 
        <h1><u>Assignment 1</u></h1>
        <h3> Quick intro + checking code works on your system </h3>
    </center>
</body>
</html>

### Learning Outcomes: The goal of this assignment is two-fold:

- This code-base is designed to be easily extended for different research projects. Running this notebook to the end will ensure that the code runs on your system, and that you are set-up to start playing with machine learning code.
- This notebook has one complete application: training a CNN classifier to predict the digit in MNIST Images. The code is written to familiarize you to a typical machine learning pipeline, and to the building blocks of code used to do ML. So, read on! 

### Please specify your Name, Email ID and forked repository url here:
- Name:
- Email:
- Link to your forked github repository:

In [20]:
### General libraries useful for python ###

import os
import sys
from tqdm import tqdm
import json
import random
import pickle
import copy
from IPython.display import display
import ipywidgets as widgets

In [2]:
### Finding where you clone your repo, so that code upstream paths can be specified programmatically ####
work_dir = os.getcwd()
git_dir = '/'.join(work_dir.split('/')[:-1])
print('Your github directory is :%s'%git_dir)

Your github directory is :/net/storage001.ib.cluster/om2/user/smadan/Harvard_BAI


In [3]:
### Libraries for visualizing our results and data ###
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
### Import PyTorch and its components ###
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim

#### Let's load our flexible code-base which you will build on for your research projects in future assignments.

Above we have imported modules (libraries for those familiar to programming languages other than python). These modules are of two kinds - (1) inbuilt python modules like `os`, `sys`, `random`, or (2) ones which we installed using conda (ex. `torch`).

Below we will be importing our own written code which resides in the `res` folder in your github directory. This is structured to be easily expandable for different machine learning projects. Suppose that you want to do a project on object detection. You can easily add a few files to the sub-folders within `res`, and this script will then be flexibly do detection instead of classication (which is presented here). Expanding on this codebase will be the main subject matter of Assignment 2. For now, let's continue with importing.

In [5]:
### Making helper code under the folder res available. This includes loaders, models, etc. ###
sys.path.append('%s/res/'%git_dir)
from models.models import get_model
from loader.loader import get_loader

Models are being loaded from: /net/storage001.ib.cluster/om2/user/smadan/Harvard_BAI/res/models
Loaders are being loaded from: /net/storage001.ib.cluster/om2/user/smadan/Harvard_BAI/res/loader


#### See those paths printed above?

`res/models` holds different model files. So, if you want to load ResNet architecture or a transformers architecture, they will reside there as separate files. 

Similarly, `res/loader` holds programs which are designed to load different types of data. For example, you may want to load data differently for object classification and detection. For classification each image will have only a numerical label corresponding to its category. For detection, the labels for the same image would contain bounding boxes for different objects and the type of the object in the box. 

So, to expand further you will be adding files to the folders above.

### Setting up Weights and Biases for tracking your experiments. ###

We have Weights and Biases (wandb.ai) integrated into the code for easy visualization of results and for tracking performance. `Please make an account at wandb.ai, and follow the steps to login to your account!`

In [7]:
import wandb
wandb.login()

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: spandanmadan (use `wandb login --relogin` to force relogin)


True

### Specifying settings/hyperparameters for our code below ###

In [8]:
wandb_config = {}
wandb_config['batch_size'] = 10
wandb_config['base_lr'] = 0.01
wandb_config['model_arch'] = 'CustomCNN'
wandb_config['num_classes'] = 10
wandb_config['run_name'] = 'TEST_1'
wandb_config['use_gpu'] = 1
wandb_config['num_epochs'] = 2
wandb_config['git_dir'] = git_dir

By changing above, different experiments can be run. For example, you can specify which model architecture to load, which dataset you will be loading, and so on.

### Data Loading ###

The most common task many of you will be doing in your projects will be running a script on a new dataset. In PyTorch this is done using data loaders, and it is extremely important to understand this works. In next assignment, you will be writing your own dataloader. For now, we only expose you to basic data loading which for the MNIST dataset for which PyTorch provides easy functions.

### Let's load MNIST. The first time you run it, the dataset gets downloaded.


Data Transforms tell PyTorch how to pre-process your data. Recall that images are stored with values between 0-255 usually. One very common pre-processing for images is to normalize to be 0 mean and 1 standard deviation. This pre-processing makes the task easier for neural networks. There are many, many kinds of normalization in deep learning, the most basic one being those imposed on the image data while loading it.

In [11]:
data_transforms = {}
data_transforms['train'] = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))])

data_transforms['test'] = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))])

`torchvision.datasets.MNIST` allows you to load MNIST data. In future, we will be using our own `get_loader` function from above to load custom data. Notice that data_transforms are passed as argument while loading the data below.

In [12]:
mnist_dataset = {}
mnist_dataset['train'] = torchvision.datasets.MNIST('%s/datasets'%wandb_config['git_dir'], train = True, download = True, transform = data_transforms['train'])
mnist_dataset['test'] = torchvision.datasets.MNIST('%s/datasets'%wandb_config['git_dir'], train = False, download = True, transform = data_transforms['test'])

#### Dataset vs Dataloader

Most deep learning datasets are huge. Can be as large as million data points. We want to keep our GPUs free to store intermediate calculations for neural networks, like gradients. We would not be able to load a million samples into the GPU (or even CPU) and do forward or backward passes on the network. 

So, samples are loaded in batches, and this method of gradient descent is called mini-batch gradient descent. `torch.utils.data.DataLoader` allows you to specify a pytorch dataset, and makes it easy to loop over it in batches. So, we leverage this to create a data loader from our above loaded MNIST dataset. 

The dataset itself only contains lists of where to find the inputs and outputs i.e. paths. The data loader defines the logic on loading this information into the GPU/CPU and so it can be passed into the neural net.

In [16]:
data_loaders = {}
data_loaders['train'] = torch.utils.data.DataLoader(mnist_dataset['train'], batch_size = wandb_config['batch_size'], shuffle = True)
data_loaders['test'] = torch.utils.data.DataLoader(mnist_dataset['test'], batch_size = wandb_config['batch_size'], shuffle = False)

data_sizes = {}
data_sizes['train'] = len(mnist_dataset['train'])
data_sizes['test'] = len(mnist_dataset['test'])

### We will use the `get_model` functionality to load a CNN architecture.

In [17]:
model = get_model(wandb_config['model_arch'], wandb_config['num_classes'])

### Curious what the model architecture looks like?

`get_model` is just a function in the file `res/models/models.py`. Stop here, open this file, and see what the function does.

In [29]:
layout = widgets.Layout(width='auto', height='90px') #set width and height

button = widgets.Button(description="Read the function?\n Click me!", layout=layout)
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("As you can see, the function simply returns an object of the class CustomCNN, which is defined in res/models/CustomCNN.py")
        print("This is our neural network model.")
button.on_click(on_button_clicked)

Button(description='Read the function?\n Click me!', layout=Layout(height='90px', width='auto'), style=ButtonS…

Output()

#### Below we have the function which trains, tests and returns the best model weights.

In [32]:
def model_pipeline(model, criterion, optimizer, dset_loaders, dset_sizes, hyperparameters):
    with wandb.init(project="HARVAR_BAI", config=hyperparameters):
        if hyperparameters['run_name']:
            wandb.run.name = hyperparameters['run_name']
        config = wandb.config
        best_model = model
        best_acc = 0.0
        
        print(config)
        
        print(config.num_epochs)
        for epoch_num in range(config.num_epochs):
            wandb.log({"Current Epoch": epoch_num})
            model = train_model(model, criterion, optimizer, dset_loaders, dset_sizes, config)
            best_acc, best_model = test_model(model, best_acc, best_model, dset_loaders, dset_sizes, config)
    
    return best_model

#### The different steps of the train model function are annotated below inside the function. Read them step by step

In [34]:
def train_model(model, criterion, optimizer, dset_loaders, dset_sizes, configs):
    print('Starting training epoch...')
    best_model = model
    best_acc = 0.0

    
    ### This tells python to track gradients. While testing weights aren't updated hence they are not stored.
    model.train() 
    running_loss = 0.0
    running_corrects = 0
    iters = 0
    
    
    ### We loop over the data loader we created above. Simply using a for loop.
    for data in tqdm(dset_loaders['train']):
        inputs, labels = data
        
        ### If you are using a gpu, then script will move the loaded data to the GPU. 
        ### If you are not using a gpu, ensure that wandb_configs['use_gpu'] is set to False above.
        if configs.use_gpu:
            inputs = inputs.float().cuda()
            labels = labels.long().cuda()
        else:
            print('WARNING: NOT USING GPU!')
            inputs = inputs.float()
            labels = labels.long()

        
        ### We set the gradients to zero, then calculate the outputs, and the loss function. 
        ### Gradients for this process are automatically calculated by PyTorch.
        
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        loss = criterion(outputs, labels)
        
        
        ### At this point, the program has calculated gradient of loss w.r.t. weights of our NN model.
        loss.backward()
        optimizer.step()
        
        ### optimizer.step() updated the models weights using calculated gradients.
        
        ### Let's store these and log them using wandb. They will be displayed in a nice online
        ### dashboard for you to see.
        
        iters += 1
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        wandb.log({"train_running_loss": running_loss/float(iters*len(labels.data))})
        wandb.log({"train_running_corrects": running_corrects/float(iters*len(labels.data))})

    epoch_loss = float(running_loss) / dset_sizes['train']
    epoch_acc = float(running_corrects) / float(dset_sizes['train'])
    wandb.log({"train_accuracy": epoch_acc})
    wandb.log({"train_loss": epoch_loss})
    return model



In [37]:
def test_model(model, best_acc, best_model, dset_loaders, dset_sizes, configs):
    print('Starting testing epoch...')
    model.eval() ### tells pytorch to not store gradients as we won't be updating weights while testing.

    running_corrects = 0
    iters = 0   
    for data in tqdm(dset_loaders['test']):
        inputs, labels = data
        if configs.use_gpu:
            inputs = inputs.float().cuda()
            labels = labels.long().cuda()
        else:
            print('WARNING: NOT USING GPU!')
            inputs = inputs.float()
            labels = labels.long()

        
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        
        iters += 1
        running_corrects += torch.sum(preds == labels.data)
        wandb.log({"train_running_corrects": running_corrects/float(iters*len(labels.data))})


    epoch_acc = float(running_corrects) / float(dset_sizes['test'])

    wandb.log({"test_accuracy": epoch_acc})
    
    ### Code is very similar to train set. One major difference, we don't update weights. 
    ### We only check the performance is best so far, if so, we save this model as the best model so far.
    
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model = copy.deepcopy(model)
    wandb.log({"best_accuracy": best_acc})
    
    return best_acc, best_model
    

In [38]:
### Criterion is simply specifying what loss to use. Here we choose cross entropy loss. 
criterion = nn.CrossEntropyLoss()

### tells what optimizer to use. There are many options, we here choose Adam.
### the main difference between optimizers is that they vary in how weights are updated based on calculated gradients.
optimizer_ft = optim.Adam(model.parameters(), lr = wandb_config['base_lr'])

if wandb_config['use_gpu']:
    criterion.cuda()
    model.cuda()

In [39]:
### Creating the folder where our models will be saved.
if not os.path.isdir("%s/saved_models/"%wandb_config['git_dir']):
    os.mkdir("%s/saved_models/"%wandb_config['git_dir'])

In [29]:
### Let's run it all, and save the final best model.


best_final_model = model_pipeline(model, criterion, optimizer_ft, data_loaders, data_sizes, wandb_config)


save_path = '%s/saved_models/%s_final.pt'%(wandb_config['git_dir'], wandb_config['run_name'])
with open(save_path,'wb') as F:
    torch.save(best_final_model,F)





  0%|          | 0/6000 [00:00<?, ?it/s]

{'batch_size': 10, 'num_classes': 10, 'model_arch': 'CustomCNN', 'base_lr': 0.01, 'num_epochs': 2, 'git_dir': '/net/storage001.ib.cluster/om2/user/smadan/Harvard_BAI', 'run_name': 'TEST_1', 'use_gpu': 1}
2
Starting training epoch...






  0%|          | 1/6000 [00:03<5:44:54,  3.45s/it]



  0%|          | 6/6000 [00:03<44:09,  2.26it/s]  



  0%|          | 17/6000 [00:03<12:24,  8.04it/s]



  0%|          | 29/6000 [00:03<06:15, 15.92it/s]



  1%|          | 41/6000 [00:03<03:55, 25.33it/s]



  1%|          | 53/6000 [00:03<02:45, 35.99it/s]



  1%|          | 65/6000 [00:04<02:04, 47.64it/s]



  1%|▏         | 79/6000 [00:04<01:34, 62.71it/s]



  2%|▏         | 91/6000 [00:04<01:20, 73.44it/s]



  2%|▏         | 105/6000 [00:04<01:07, 86.98it/s]



  2%|▏         | 119/6000 [00:04<00:59, 98.44it/s]



  2%|▏         | 133/6000 [00:04<00:54, 107.38it/s]



  2%|▏         | 147/6000 [00:04<00:51, 114.56it/s]



  3%|▎         | 160/6000 [00:04<00:49, 117.97it/s]



  3%|▎         | 174/6000 [00:04<00:47, 122.00it/s]



  3%|▎         | 188/6000 [00:05<00:46, 124.96it/s]



  3%|▎         | 202/6000 [00:05<00:45, 127.28it/s]



  4%|▎         | 216/6000 [00:05<00:44, 129.23it/s]



  4%|▍         | 230/600

 28%|██▊       | 1671/6000 [00:16<00:32, 132.20it/s]



 28%|██▊       | 1685/6000 [00:16<00:32, 132.29it/s]



 28%|██▊       | 1699/6000 [00:16<00:32, 132.58it/s]



 29%|██▊       | 1713/6000 [00:16<00:32, 132.86it/s]



 29%|██▉       | 1727/6000 [00:16<00:32, 132.84it/s]



 29%|██▉       | 1741/6000 [00:16<00:32, 132.77it/s]



 29%|██▉       | 1755/6000 [00:16<00:32, 132.51it/s]



 29%|██▉       | 1769/6000 [00:17<00:32, 129.26it/s]



 30%|██▉       | 1782/6000 [00:17<00:32, 128.41it/s]



 30%|██▉       | 1795/6000 [00:17<00:32, 128.81it/s]



 30%|███       | 1808/6000 [00:17<00:32, 129.09it/s]



 30%|███       | 1822/6000 [00:17<00:32, 129.52it/s]



 31%|███       | 1835/6000 [00:17<00:32, 129.59it/s]



 31%|███       | 1848/6000 [00:17<00:32, 129.12it/s]



 31%|███       | 1861/6000 [00:17<00:32, 128.81it/s]



 31%|███       | 1874/6000 [00:17<00:32, 128.43it/s]



 31%|███▏      | 1887/6000 [00:17<00:31, 128.86it/s]



 32%|███▏      | 1901/6000 [00:18<00:31, 129.55i

 55%|█████▍    | 3285/6000 [00:29<00:20, 131.90it/s]



 55%|█████▍    | 3299/6000 [00:29<00:20, 131.91it/s]



 55%|█████▌    | 3313/6000 [00:29<00:20, 130.84it/s]



 55%|█████▌    | 3327/6000 [00:29<00:20, 131.22it/s]



 56%|█████▌    | 3341/6000 [00:29<00:20, 131.49it/s]



 56%|█████▌    | 3355/6000 [00:29<00:20, 132.02it/s]



 56%|█████▌    | 3369/6000 [00:29<00:19, 132.74it/s]



 56%|█████▋    | 3383/6000 [00:29<00:19, 133.15it/s]



 57%|█████▋    | 3397/6000 [00:29<00:19, 133.15it/s]



 57%|█████▋    | 3411/6000 [00:29<00:19, 132.97it/s]



 57%|█████▋    | 3425/6000 [00:30<00:19, 132.10it/s]



 57%|█████▋    | 3439/6000 [00:30<00:19, 130.65it/s]



 58%|█████▊    | 3453/6000 [00:30<00:19, 130.34it/s]



 58%|█████▊    | 3467/6000 [00:30<00:19, 130.79it/s]



 58%|█████▊    | 3481/6000 [00:30<00:19, 131.60it/s]



 58%|█████▊    | 3495/6000 [00:30<00:18, 132.54it/s]



 58%|█████▊    | 3509/6000 [00:30<00:18, 133.20it/s]



 59%|█████▊    | 3523/6000 [00:30<00:18, 133.29i

...

 81%|████████  | 4831/6000 [00:38<00:09, 118.13it/s]



 81%|████████  | 4844/6000 [00:38<00:09, 120.19it/s]



 81%|████████  | 4857/6000 [00:38<00:09, 121.43it/s]



 81%|████████  | 4870/6000 [00:38<00:09, 122.34it/s]



 81%|████████▏ | 4883/6000 [00:38<00:09, 123.17it/s]



 82%|████████▏ | 4896/6000 [00:38<00:08, 124.09it/s]



 82%|████████▏ | 4909/6000 [00:38<00:08, 124.46it/s]



 82%|████████▏ | 4922/6000 [00:38<00:08, 124.78it/s]



 82%|████████▏ | 4935/6000 [00:39<00:08, 125.20it/s]



 82%|████████▏ | 4948/6000 [00:39<00:08, 125.82it/s]



 83%|████████▎ | 4961/6000 [00:39<00:08, 125.49it/s]



 83%|████████▎ | 4974/6000 [00:39<00:08, 125.27it/s]



 83%|████████▎ | 4987/6000 [00:39<00:08, 125.59it/s]



 83%|████████▎ | 5000/6000 [00:39<00:07, 125.94it/s]



 84%|████████▎ | 5013/6000 [00:39<00:07, 126.19it/s]



 84%|████████▍ | 5026/6000 [00:39<00:07, 126.57it/s]



 84%|████████▍ | 5039/6000 [00:39<00:07, 126.53it/s]



 84%|████████▍ | 5052/6000 [00:39<00:07, 126.10i

Starting testing epoch...






  5%|▌         | 52/1000 [00:00<00:03, 252.73it/s]



  8%|▊         | 78/1000 [00:00<00:03, 249.72it/s]



 10%|█         | 103/1000 [00:00<00:03, 247.76it/s]



 13%|█▎        | 128/1000 [00:00<00:03, 247.70it/s]



 15%|█▌        | 153/1000 [00:00<00:03, 247.36it/s]



 18%|█▊        | 178/1000 [00:00<00:03, 247.86it/s]



 20%|██        | 203/1000 [00:00<00:03, 248.43it/s]



 23%|██▎       | 228/1000 [00:00<00:03, 246.22it/s]



 25%|██▌       | 253/1000 [00:01<00:03, 243.56it/s]



 28%|██▊       | 278/1000 [00:01<00:02, 241.48it/s]



 30%|███       | 303/1000 [00:01<00:02, 240.54it/s]



 33%|███▎      | 328/1000 [00:01<00:02, 239.59it/s]



 35%|███▌      | 353/1000 [00:01<00:02, 240.33it/s]



 38%|███▊      | 378/1000 [00:01<00:02, 242.28it/s]



 40%|████      | 403/1000 [00:01<00:02, 242.69it/s]



 43%|████▎     | 428/1000 [00:01<00:02, 242.97it/s]



 45%|████▌     | 453/1000 [00:01<00:02, 243.85it/s]



 48%|████▊     | 478/1000 [00:01<00:02, 242.60it/s]



 50%|███

train_running_corrects,0.6257
train_accuracy,0.5193
train_running_loss,0.19037
_runtime,119
test_accuracy,0.6257
best_accuracy,0.6257
train_loss,0.19037
_timestamp,1611879518
_step,26009
Current Epoch,1


train_running_corrects,▁▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▄▄▄▅▄▅▅▅▅▅▅▅▅▅▅▅▅▇██
train_running_loss,█▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁█
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
best_accuracy,▁█
train_loss,█▁
Current Epoch,▁█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁█


### Congratulations!

You just completed your first deep learning program - image classification for MNIST. This wraps up assignment 1. In the next assignment, we will see how you can make changes to above mentioned folders/files to adapt this code-base to your own research project.

# Deliverables for Assignment 1: 

### Please run this assignment through to the end, and then make two submissions:

- Download this notebook as an HTML file. Click File ---> Download as ---> HTML. Submit this on canvas.
- Add, commit and push these changes to your github repository.